# Tic Tac Toe
Aneta Postrożny
___

### Gra w kółko i krzyżyk:
Gra w kółko i krzyżyk jest jedną z najprostszych gier planszowych. Grając w kółko i krzyżyk, gracze zajmują pola na planszy, a następnie sprawdzają, czy udało im się ułożyć trzy znaki w rzędzie, w kolumnie lub na skos. Wygrywa ten, kto pierwszy ułoży trzy znaki w rzędzie, w kolumnie lub na skos. Jeśli plansza zostanie wypełniona, a żaden z graczy nie ułożył trzech znaków w rzędzie, w kolumnie lub na skos, to gra kończy się remisem.
___

### Cel ćwiczenia
Celem ćwiczenia jest napisanie prostego programu, który będzie grał w kółko i krzyżyk z innym graczem. Probabilistyczna wersja gry ma zakładać 20% prawdopodobieństwo, że ruch się nie udaje.
___

### Rozwiązanie
Rozwiązanie zostało oparte na kodzie zamieszczonym na MS Teams. Rozwiązanie zostało napisane w języku Python w środowisku Jupyter z wykorzystaniem biblioteki easyAI. Kod został zmodyfikowany w celu zaimplementowania probabilistycznego wariantu gry. Dodatkowo zostały dodane statystyki, które pozwolą na porównanie dwóch różnych ustawień maksymalnej głębokości dla gier deterministycznych na deterministycznym i probabilistycznym wariancie gry. Zastosowano algorytm Negamax, który jest algorytmem minimax z podstawową heurystyką. Algorytm Negamax jest algorytmem optymalnym, który zawsze znajduje najlepszy ruch. Algorytm Negamax jest algorytmem rekurencyjnym, który zawsze wybiera najlepszy ruch z punktu widzenia gracza, który wykonuje ruch.
___

In [1]:
# !pip install easyAI

In [2]:
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax

import numpy as np
import time
import random

### Plansza

In [3]:
class Board:
    def __init__(self):
        # start with en empty board
        self.state = [[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]

    def show(self):
        print("Current board: ")
        for i in range(3):
            print(self.state[i])

### Gra

In [4]:
class TicTacToe(TwoPlayerGame):
    """
    In turn, the players leave their mark ('X' or 'O') on a 3x3 board.
    The player who places three of their symbols in a row, column or diagonal
    wins.
    """

    def __init__(self, players, deterministic=True, player=1):
        self.players = players
        self.board = Board()
        self.current_player = player  # player 1 starts
        self.deterministic = deterministic
        self.nmoves_player1 = 0
        self.nmoves_player2 = 0
        self.time_player1 = 0
        self.time_player2 = 0

    def player_mark(self, nplayer):
        if nplayer == 1:
            return 'X'
        else:
            return 'O'

    def possible_moves(self):
        moves = []
        for i in range(3):
            for j in range(3):
                if self.board.state[i][j] == ' ':
                    moves.append((i, j))
        # print("Possible moves: ", moves)
        return moves

    def make_move(self, move):
        time_start = time.time()
        if self.deterministic:
            self.board.state[move[0]][move[1]] = self.player_mark(self.current_player)
            # count number of moves and time for each player
            if self.current_player == 1:
                self.nmoves_player1 += 1
                self.time_player1 += time.time() - time_start
            else:
                self.nmoves_player2 += 1
                self.time_player2 += time.time() - time_start
        else:
            if random.random() < 0.8: # Possibility of making move is 80%
                self.board.state[move[0]][move[1]] = self.player_mark(self.current_player)
                if self.current_player == 1:
                    self.nmoves_player1 += 1
                    self.time_player1 += time.time() - time_start
                else:
                    self.nmoves_player2 += 1
                    self.time_player2 += time.time() - time_start

    def winner(self):
        for p in range(1, 3):
            # Do we have three same symbols in a row?
            cur_symbol = self.player_mark(p)
            for i in range(3):
                all_in_row = True
                for j in range(3):
                    if self.board.state[i][j] != cur_symbol:
                        all_in_row = False
                if all_in_row:
                    return p  # p won

            # Do we have three same symbols in a column?
            for i in range(3):
                all_in_col = True
                for j in range(3):
                    if self.board.state[j][i] != cur_symbol:
                        all_in_col = False
                if all_in_col:
                    return p  # p won

            # Do we have three same symbols in a diagonal?
            all_in_l_diag = True
            for j in range(3):
                if self.board.state[j][j] != cur_symbol:
                    all_in_l_diag = False
            if all_in_l_diag:
                return p  # p won

            all_in_r_diag = True
            for j in range(3):
                if self.board.state[2 - j][j] != cur_symbol:
                    all_in_r_diag = False
            if all_in_r_diag:
                return p  # p won

        return None

    def is_over(self):
        return (self.possible_moves() == []) or (self.winner() is not None)

    def show(self):
        self.board.show()

    def scoring(self):
        won = self.winner()
        if won == self.current_player:
            return 100
        elif won == 3 - self.current_player:
            return -100
        else:
            return 0

### Algorytm Negamax bez obcięcia alfa-beta

In [5]:
inf = float("infinity")


def negamax_without_ab(game, depth, origDepth, scoring):
    if (depth == 0) or game.is_over():
        return scoring(game) * (1 + 0.001 * depth)

    possible_moves = game.possible_moves()

    state = game
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    best_value = -inf
    unmake_move = hasattr(state, "unmake_move")

    for move in possible_moves:
        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_alpha = -negamax_without_ab(game, depth - 1, origDepth, scoring)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        if best_value < move_alpha:
            best_value = move_alpha

        if best_value == inf:
            break

    return best_value


class NegamaxWithoutAB:

    def __init__(self, depth, scoring=None, win_score=+inf):
        self.scoring = scoring
        self.depth = depth
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """

        scoring = (
            self.scoring if self.scoring else (lambda g: g.scoring())
        )  # horrible hack

        self.alpha = negamax_without_ab(
            game,
            self.depth,
            self.depth,
            scoring,
        )
        return game.ai_move


### Zadania
1. Napisz kod, który uruchamia dwóch graczy AI z algorytmem Negamax przeciwko sobie wielokrotnie, zmieniając gracza rozpoczynającego. Policz liczbę zwycięstw każdego z graczy. Porównaj dwa różne ustawienia maksymalnej głębokości dla gier deterministycznych na deterministycznym i probabilistycznym wariancie Twojej gry.
2. Napisz kod, który mierzy średni czas spędzony na wybieraniu akcji przez każdy wariant AI. Dodaj zmierzone czasy do raportu.
3. Porównaj kilka (co najmniej Negamax z i bez odcięcia alfa-beta, z dwoma różnymi ustawienia maksymalnej głębokości) algorytmy dla gier deterministycznych na deterministycznych i probabilistycznych wariantach twojej gry.

In [6]:
def games_with_stats(depth1=4, depth2=4, ngames=10, deterministic=True, with_ab=True):

    if with_ab:
        ai = Negamax(depth1)
        ai2 = Negamax(depth2)
    else:
        ai = NegamaxWithoutAB(depth1)
        ai2 = NegamaxWithoutAB(depth2)

    stats = {
        'player1': {
            'wins': 0,
            'losses': 0,
            'ties': 0,
            'avg_action_time': 0
        },
        'player2': {
            'wins': 0,
            'losses': 0,
            'ties': 0,
            'avg_action_time': 0
        }
    }

    # Use a flag variable to keep track of which player started the previous game
    player1_starts = True

    # Keep track of the times for each player
    player1_times = []
    player2_times = []

    for i in range(1, ngames + 1):
        # Determine the players for the game
        players = [AI_Player(ai), AI_Player(ai2)]
        if player1_starts:
            player = 1
        else:
            player = 2

        # Play the game
        game = TicTacToe(players, deterministic=deterministic, player=player)
        print("------------------------------------------------------------")
        print("Starting game ", i)
        game.play()
        print("Game ", i, " finished. Winner: ", game.winner())
        print("------------------------------------------------------------")

        # Update the times (append avg time per move)
        if game.nmoves_player1 > 0:
            player1_times.append(game.time_player1/game.nmoves_player1)
        if game.nmoves_player2 > 0:
            player2_times.append(game.time_player2/game.nmoves_player2)

        # Update the stats
        if game.winner() == 1:
            stats['player1']['wins'] += 1
            stats['player2']['losses'] += 1
        elif game.winner() == 2:
            stats['player1']['losses'] += 1
            stats['player2']['wins'] += 1
        else:
            stats['player1']['ties'] += 1
            stats['player2']['ties'] += 1

        # Update the flag variable
        player1_starts = not player1_starts

    # Update the stats with the average times
    stats['player1']['avg_action_time'] = sum(player1_times)/len(player1_times)
    stats['player2']['avg_action_time'] = sum(player2_times)/len(player2_times)

    print("********************************************************************")
    print("Stats: ")
    print(stats)
    print("********************************************************************")

### Porównanie wyników dla takich samych głębokości dla gier deterministycznych i probabilistycznych

- Rozgrywka 10 gier z taką samą głębokością dla obu graczy w grze probabilistycznej z Negamaxem z odcięciem alfa-beta

In [7]:
games_with_stats(depth1=4, depth2=4, ngames=10, deterministic=False)

------------------------------------------------------------
Starting game  1
Current board: 
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #1: player 1 plays (0, 0) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #2: player 2 plays (0, 1) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #3: player 1 plays (0, 1) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #4: player 2 plays (0, 1) :
Current board: 
['X', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #5: player 1 plays (1, 0) :
Current board: 
['X', 'O', ' ']
['X', ' ', ' ']
[' ', ' ', ' ']

Move #6: player 2 plays (2, 0) :
Current board: 
['X', 'O', ' ']
['X', ' ', ' ']
[' ', ' ', ' ']

Move #7: player 1 plays (2, 0) :
Current board: 
['X', 'O', ' ']
['X', ' ', ' ']
[' ', ' ', ' ']

Move #8: player 2 plays (0, 2) :
Current board: 
['X', 'O', 'O']
['X', ' ', ' ']
[' ', ' ', ' ']

Move #9: player 1 plays (1, 1) :
Current board: 
['X', 'O', 'O']
['X', ' 

- Rozgrywka 10 gier z taką samą głębokością dla obu graczy w grze deterministycznej z Negamaxem z odcięciem alfa-beta

In [8]:
games_with_stats(depth1=4, depth2=4, ngames=10, deterministic=True)

------------------------------------------------------------
Starting game  1
Current board: 
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #1: player 1 plays (0, 0) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #2: player 2 plays (0, 1) :
Current board: 
['X', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #3: player 1 plays (0, 2) :
Current board: 
['X', 'O', 'X']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #4: player 2 plays (1, 1) :
Current board: 
['X', 'O', 'X']
[' ', 'O', ' ']
[' ', ' ', ' ']

Move #5: player 1 plays (2, 1) :
Current board: 
['X', 'O', 'X']
[' ', 'O', ' ']
[' ', 'X', ' ']

Move #6: player 2 plays (1, 0) :
Current board: 
['X', 'O', 'X']
['O', 'O', ' ']
[' ', 'X', ' ']

Move #7: player 1 plays (1, 2) :
Current board: 
['X', 'O', 'X']
['O', 'O', 'X']
[' ', 'X', ' ']

Move #8: player 2 plays (2, 2) :
Current board: 
['X', 'O', 'X']
['O', 'O', 'X']
[' ', 'X', 'O']

Move #9: player 1 plays (2, 0) :
Current board: 
['X', 'O', 'X']
['O', 'O

- Rozgrywka 10 gier z taką samą głębokością dla obu graczy w grze probabilistycznej z Negamaxem bez odcięcia alfa-beta

In [9]:
games_with_stats(depth1=4, depth2=4, ngames=10, deterministic=False, with_ab=False)

------------------------------------------------------------
Starting game  1
Current board: 
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #1: player 1 plays (0, 0) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #2: player 2 plays (0, 1) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #3: player 1 plays (0, 1) :
Current board: 
['X', 'X', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #4: player 2 plays (0, 2) :
Current board: 
['X', 'X', 'O']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #5: player 1 plays (1, 0) :
Current board: 
['X', 'X', 'O']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #6: player 2 plays (1, 0) :
Current board: 
['X', 'X', 'O']
['O', ' ', ' ']
[' ', ' ', ' ']

Move #7: player 1 plays (1, 1) :
Current board: 
['X', 'X', 'O']
['O', 'X', ' ']
[' ', ' ', ' ']

Move #8: player 2 plays (1, 2) :
Current board: 
['X', 'X', 'O']
['O', 'X', 'O']
[' ', ' ', ' ']

Move #9: player 1 plays (2, 0) :
Current board: 
['X', 'X', 'O']
['O', 'X

- Rozgrywka 10 gier z taką samą głębokością dla obu graczy w grze deterministycznej z Negamaxem bez odcięcia alfa-beta

In [ ]:
games_with_stats(depth1=4, depth2=4, ngames=10, deterministic=True, with_ab=False)

------------------------------------------------------------
Starting game  1
Current board: 
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #1: player 1 plays (0, 0) :
Current board: 
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #2: player 2 plays (0, 1) :
Current board: 
['X', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #3: player 1 plays (0, 2) :
Current board: 
['X', 'O', 'X']
[' ', ' ', ' ']
[' ', ' ', ' ']

Move #4: player 2 plays (1, 0) :
Current board: 
['X', 'O', 'X']
['O', ' ', ' ']
[' ', ' ', ' ']

Move #5: player 1 plays (1, 1) :
Current board: 
['X', 'O', 'X']
['O', 'X', ' ']
[' ', ' ', ' ']

Move #6: player 2 plays (1, 2) :
Current board: 
['X', 'O', 'X']
['O', 'X', 'O']
[' ', ' ', ' ']

Move #7: player 1 plays (2, 0) :
Current board: 
['X', 'O', 'X']
['O', 'X', 'O']
['X', ' ', ' ']
Game  1  finished. Winner:  1
------------------------------------------------------------
------------------------------------------------------------
Starting game  2
Cur

Przykładowe wyniki:
- Wersja probabilistyczna:
    - player1:
        - wins: 5,
        - losses: 3,
        - ties: 2,
    - player2:
        - wins: 3,
        - losses: 5,
        - ties: 2
- Wersja deterministyczna:
    - player1:
        - wins: 0,
        - losses: 0,
        - ties: 10,
    - player2:
        - wins: 0,
        - losses: 0,
        - ties: 10
- Wersja probabilistyczna bez odcięcia alfa-beta:
    - player1:
        - wins: 3,
        - losses: 4,
        - ties: 3,
    - player2:
        - wins: 4,
        - losses: 3,
        - ties: 3
- Wersja deterministyczna:
    - player1:
        - wins: 5,
        - losses: 5,
        - ties: 0,
    - player2:
        - wins: 5,
        - losses: 5,
        - ties: 0

Średni czas wykonania ruchu dla każdego z graczy na podstawie 10 gier:
- Wersja probabilistyczna:
    - player1:
        - avg_action_time: 0.00000056.
    - player2:
        - avg_action_time: 0.00000048
- Wersja deterministyczna:
    - player1:
        - avg_action_time: 0.00000041.
    - player2:
        - avg_action_time: 00.00000053
- Wersja probabilistyczna bez odcięcia alfa-beta:
    - player1:
        - avg_action_time: 0.00000062.
    - player2:
        - avg_action_time: 0.00000060
- Wersja deterministyczna bez odcięcia alfa-beta:
    - player1:
        - avg_action_time: 0.00000058.
    - player2:
        - avg_action_time: 00.00000054

Wnioski:
- W przypadku gry deterministycznej, obaj gracze mają taką samą głębokość przewidywania ruchów, więc gra zawsze kończy się remisem.
- W przypadku gry probabilistycznej, obaj gracze mają taką samą głębokość przewidywania ruchów, ale za razem, każdy z ich ruchów ma szansę na 80% na powodzenie. Dlatego w przypadku gry probabilistycznej, obaj gracze mają szansę na zwycięstwo. Po kilkukrotnym uruchomieniu gry, zaobserwować można, że obaj gracze mają około 50% szans na zwycięstwo, a remisy nie pojawiają się aż tak często.

### Porównanie wyników dla różnych głębokości dla gier deterministycznych i probabilistycznych

- Rozgrywka 10 gier z różnymi głębokościami dla obu graczy w grze probabilistycznej

In [ ]:
games_with_stats(depth1=4, depth2=6, ngames=10, deterministic=False)

- Rozgrywka 10 gier z różnymi głębokościami dla obu graczy w grze deterministycznej

In [ ]:
games_with_stats(depth1=4, depth2=6, ngames=10, deterministic=True)

- Rozgrywka 10 gier z różnymi głębokościami dla obu graczy w grze probabilistycznej z Negamaxem bez odcięcia alfa-beta

In [ ]:
games_with_stats(depth1=4, depth2=6, ngames=10, deterministic=False, with_ab=False)

- Rozgrywka 10 gier z różnymi głębokościami dla obu graczy w grze deterministycznej z Negamaxem bez odcięcia alfa-beta

In [ ]:
games_with_stats(depth1=4, depth2=6, ngames=10, deterministic=True, with_ab=False)

Przykładowe wyniki:
- Wersja probabilistyczna:
    - player1:
        - wins: 2,
        - losses: 6,
        - ties: 2,
    - player2:
        - wins: 6,
        - losses: 2,
        - ties: 2
- Wersja deterministyczna:
    - player1:
        - wins: 0,
        - losses: 5,
        - ties: 5,
    - player2:
        - wins: 5,
        - losses: 0,
        - ties: 5
- Wersja probabilistyczna bez odcięcia alfa-beta:
    - player1:
        - wins: 4,
        - losses: 2,
        - ties: 4,
    - player2:
        - wins: 2,
        - losses: 4,
        - ties: 4
- Wersja deterministyczna bez odcięcia alfa-beta:
    - player1:
        - wins: 5,
        - losses: 5,
        - ties: 0,
    - player2:
        - wins: 5,
        - losses: 5,
        - ties: 0

Średni czas wykonania ruchu dla każdego z graczy na podstawie 10 gier:
- Wersja probabilistyczna:
    - player1:
        - avg_action_time: 0.00000065.
    - player2:
        - avg_action_time: 0.00000057
- Wersja deterministyczna:
    - player1:
        - avg_action_time: 0.00000059.
    - player2:
        - avg_action_time: 00.00000052
- Wersja probabilistyczna bez odcięcia alfa-beta:
    - player1:
        - avg_action_time: 0.00000056.
    - player2:
        - avg_action_time: 0.00000062
- Wersja deterministyczna bez odcięcia alfa-beta:
    - player1:
        - avg_action_time: 0.00000059.
    - player2:
        - avg_action_time: 00.00000083

Wnioski:
- W przypadku gry deterministycznej, obaj gracze mają różną głębokość przewidywania ruchów, w zastosowanej przeze mnie konfiguracji dochodzi zawsze do równej ilości zwycięstw dla gracza o większej głębokości przewidywania, co ilości remisów. Remisy zdarzają się wtedy, gdy gracz o mniejszej głębokości przewidywania rozpoczyna grę.
- W przypadku gry probabilistycznej, jak można się domyślić większą liczbę wygranych osiąga gracz o większej głębokości przewidywania ruchów. Remisy występują jednakże w mniejszej ilości niż w przypadku gry deterministycznej. Różnicą jest to, że gracz o mniejszej głębokości przewidywania ruchów wciąż ma szanse na wygraną.